## Assignment : 14

<pre>
1. You can work with preprocessed_data.csv for the assignment. You can get the data from - <a href='https://drive.google.com/drive/u/0/folders/1CJnItndeSSJu7aragQoXWZS9-0apN6pp'>Data folder </a>
2. Load the data in your notebook.
3. After step 2 you have to train 3 types of models as discussed below. 
4. For all the model use <a href='https://scikit-learn.org/stable/modules/model_evaluation.html#roc-metrics'>'auc'</a> as a metric. check <a  href='https://stackoverflow.com/a/46844409'>this</a> and <a  href='https://www.kaggle.com/c/santander-customer-transaction-prediction/discussion/80807'>this</a> for using auc as a metric 
5. You are free to choose any number of layers/hiddden units but you have to use same type of architectures shown below. 
6. You can use any one of the optimizers and choice of Learning rate and momentum.
7. For all the model's use <a href='https://www.youtube.com/watch?v=2U6Jl7oqRkM'>TensorBoard</a> and plot the Metric value and Loss with epoch. While submitting, take a screenshot of plots and include those images in a separate pad and write your observations about them.
8. Make sure that you are using GPU to train the given models.
</pre>

In [ ]:
#you can use gdown modules to import dataset for the assignment
#for importing any file from drive to Colab you can write the syntax as !gdown --id file_id
#you can run the below cell to import the required preprocessed data.csv file and glove vector

In [ ]:
#!gdown --id 1GpATd_pM4mcnWWIs28-s1lgqdAg2Wdv-
#!gdown --id 1pGd5tLwA30M7wkbJKdXHaae9tYVDICJ_

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from keras import Model,Input
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM, SpatialDropout1D, BatchNormalization,concatenate,Flatten,Embedding,Dense,Dropout,MaxPooling2D,Reshape, Conv1D
from keras.layers.convolutional import Conv2D,Conv1D
from sklearn.preprocessing import StandardScaler
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint,LearningRateScheduler
from keras.initializers import he_normal
from tensorflow.python.keras.callbacks import TensorBoard
import warnings
warnings.filterwarnings("ignore")
import os
from keras.preprocessing.sequence import pad_sequences
import pickle
from sklearn.preprocessing import LabelEncoder

## <font color='red'> Model-1 </font>
Build and Train deep neural network as shown below

<img src='https://i.imgur.com/w395Yk9.png'>
ref: https://i.imgur.com/w395Yk9.png

- __Input_seq_total_text_data__ --- You have to give Total text data columns. After this use the Embedding layer to get word vectors. Use given predefined glove word vectors, don't train any word vectors. After this use LSTM and get the LSTM output and Flatten that output. 
- __Input_school_state__ --- Give 'school_state' column as input to embedding layer and Train the Keras Embedding layer. 
- __Project_grade_category__  --- Give 'project_grade_category' column as input to embedding layer and Train the Keras Embedding layer.
- __Input_clean_categories__ --- Give 'input_clean_categories' column as input to embedding layer and Train the Keras Embedding layer.
- __Input_clean_subcategories__ --- Give 'input_clean_subcategories' column as input to embedding layer and Train the Keras Embedding layer.
- __Input_clean_subcategories__ --- Give 'input_teacher_prefix' column as input to embedding layer and Train the Keras Embedding layer.
- __Input_remaining_teacher_number_of_previously_posted_projects._resource_summary_contains_numerical_digits._price._quantity__ ---concatenate remaining columns and add a Dense layer after that. 



Below is an example of embedding layer for a categorical columns. In below code all are dummy values, we gave only for referance. 

In [ ]:
# # https://stats.stackexchange.com/questions/270546/how-does-keras-embedding-layer-work
# input_layer = Input(shape=(n,))
# embedding = Embedding(no_1, no_2, input_length=n)(input_layer)
# flatten = Flatten()(embedding)

### 1. Go through this blog, if you have any doubt on using predefined Embedding values in Embedding layer - https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
### 2. Please go through this link https://keras.io/getting-started/functional-api-guide/ and check the 'Multi-input and multi-output models' then you will get to know how to give multiple inputs. 

# <font color='red'> Model-1 </font>

In [ ]:
os.chdir(r'D:\Applied_AI\Ass_26_LSTM')

In [ ]:
# import all the libraries
#make sure that you import your libraries from tf.keras and not just keras
import tensorflow as tf
from tensorflow.keras.layers import Input,Dense,LSTM

In [3]:
#read the csv file
import pandas as pd
# df = pd.read_csv('preprocessed_data.csv')
df = pd.read_csv('../input/preprocesseddonorchoose/preprocessed_data.csv')
df.head(2)

In [4]:
X = df.drop(['project_is_approved'], axis=1)
y = df['project_is_approved']
X.shape, y.shape

In [5]:
# perform stratified train test split on the dataset
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.25, stratify=y)
X_tr.shape, X_te.shape, y_tr.shape, y_te.shape

## 1.1 Text Vectorization

In [6]:
#since the data is already preprocessed, we can directly move to vectorization part
#first we will vectorize the text data
#for vectorization of text data in deep learning we use tokenizer, you can go through below references
# https://www.kdnuggets.com/2020/03/tensorflow-keras-tokenization-text-data-prep.html
#https://stackoverflow.com/questions/51956000/what-does-keras-tokenizer-method-exactly-do
# after text vectorization you should get train_padded_docs and test_padded_docs

In [7]:
num_words = 1000
oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'

In [8]:
train_data = X_tr['essay']
# Tokenize our training data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data)

# Get our training data word index
word_index = tokenizer.word_index

# Encode training data sentences into sequences
train_sequences = tokenizer.texts_to_sequences(train_data)

# Get max training sequence length
maxlen = 300
# Pad the training sequences
train_padded = pad_sequences(train_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)
pad_len = train_padded.shape[1]

In [9]:
test_data = X_te['essay']
test_sequences = tokenizer.texts_to_sequences(test_data)
test_padded = pad_sequences(test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

In [10]:
train_padded.shape,test_padded.shape

In [ ]:
#after getting the padded_docs you have to use predefined glove vectors to get 300 dim representation for each word
# we will be storing this data in form of an embedding matrix and will use it while defining our model
# Please go through following blog's 'Example of Using Pre-Trained GloVe Embedding' section to understand how to create embedding matrix
# https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

In [11]:
with open('../input/donor-choose-preprocessed/glove_vectors', 'rb') as f:
    glov = pickle.load(f)
    g_words =  set(glov.keys())

In [12]:
# create a weight matrix for words in training docs
vocab_size = len(word_index)+1
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in tokenizer.word_index.items():
	if word in g_words:
		embedding_vector = glov[word]
		embedding_matrix[i] = embedding_vector
embedding_matrix.shape

In [13]:
vocab_size

## 1.2 Categorical feature Vectorization

In [ ]:
# for model 1 and model 2, we have to assign a unique number to each feature in a particular categorical column.
# you can either use tokenizer,label encoder or ordinal encoder to perform the task
# label encoder gives an error for 'unseen values' (values present in test but not in train)
# handle unseen values with label encoder - https://stackoverflow.com/a/56876351
# ordinal encoder also gives error with unseen values but you can use modify handle_unknown parameter
# documentation of ordianl encoder https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html
# after categorical feature vectorization you will have column_train_data and column_test_data.


In [14]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
encoder.fit(X_tr['school_state'].values.reshape(-1,1))
X_tr_ss = encoder.transform(X_tr['school_state'].values.reshape(-1,1))
X_te_ss = encoder.transform(X_te['school_state'].values.reshape(-1,1))

In [15]:
encoder.fit(X_tr['teacher_prefix'].values.reshape(-1,1))
X_tr_tp = encoder.transform(X_tr['teacher_prefix'].values.reshape(-1,1))
X_te_tp = encoder.transform(X_te['teacher_prefix'].values.reshape(-1,1))

In [16]:
encoder.fit(X['clean_categories'].values.reshape(-1,1))
X_tr_cc = encoder.transform(X_tr['clean_categories'].values.reshape(-1,1))
X_te_cc = encoder.transform(X_te['clean_categories'].values.reshape(-1,1))

In [17]:
encoder.fit(X['clean_subcategories'].values.reshape(-1,1))
X_tr_csc = encoder.transform(X_tr['clean_subcategories'].values.reshape(-1,1))
X_te_csc = encoder.transform(X_te['clean_subcategories'].values.reshape(-1,1))

In [19]:
encoder.fit(X['project_grade_category'].values.reshape(-1,1))
X_tr_pgc = encoder.transform(X_tr['project_grade_category'].values.reshape(-1,1))
X_te_pgc = encoder.transform(X_te['project_grade_category'].values.reshape(-1,1))

In [20]:
print('Encoded train school_state shape',X_tr_ss.shape)
print('Encoded train teacher_prefix shape',X_tr_tp.shape)
print('Encoded train clean_categories shape',X_tr_cc.shape)
print('Encoded train clean_subcategories shape',X_tr_csc.shape)
print('Encoded train project_grade_category shape',X_tr_pgc.shape)

In [21]:
print('Encoded test school_state shape',X_te_ss.shape)
print('Encoded test teacher_prefix shape',X_te_tp.shape)
print('Encoded test clean_categories shape',X_te_cc.shape)
print('Encoded test clean_subcategories shape',X_te_csc.shape)
print('Encoded test project_grade_category shape',X_te_pgc.shape)

In [22]:
catagorical_data_train = np.hstack((X_tr_ss,X_tr_tp,X_tr_cc,X_tr_csc,X_tr_pgc))
catagorical_data_test = np.hstack((X_te_ss,X_te_tp,X_te_cc,X_te_csc,X_te_pgc))
catagorical_data_train.shape,catagorical_data_test.shape


## 1.3 Numerical feature Vectorization

In [ ]:
# you have to standardise the numerical columns
# stack both the numerical features
#after numerical feature vectorization you will have numerical_data_train and numerical_data_test

In [23]:
standardizer = StandardScaler()
standardizer.fit(X_tr['teacher_number_of_previously_posted_projects'].values.reshape(-1,1))
X_tr_np = standardizer.transform(X_tr['teacher_number_of_previously_posted_projects'].values.reshape(-1,1))
X_te_np = standardizer.transform(X_te['teacher_number_of_previously_posted_projects'].values.reshape(-1,1))

In [24]:
standardizer.fit(X_tr['price'].values.reshape(-1,1))
X_tr_price = standardizer.transform(X_tr['price'].values.reshape(-1,1))
X_te_price = standardizer.transform(X_te['price'].values.reshape(-1,1))

In [25]:
print('Encoded train teacher_number_of_previously_posted_projects shape',X_tr_np.shape)
print('Encoded train price shape',X_tr_price.shape)

In [26]:
print('Encoded test teacher_number_of_previously_posted_projects shape',X_te_np.shape)
print('Encoded test price shape',X_te_price.shape)

In [27]:
numerical_data_train = np.hstack((X_tr_np,X_tr_price))
numerical_data_test = np.hstack((X_te_np,X_te_price))

numerical_data_train.shape,numerical_data_test.shape


In [28]:
X_tr_fin1 = [train_padded,X_tr_ss,X_tr_tp,X_tr_cc,X_tr_csc,X_tr_pgc,numerical_data_train]
X_te_fin1 = [test_padded,X_te_ss,X_te_tp,X_te_cc,X_te_csc,X_te_pgc,numerical_data_test]

In [29]:
# https://www.geeksforgeeks.org/python-keras-keras-utils-to_categorical/
from keras.utils import np_utils
y_tr_fin = np_utils.to_categorical(y_tr, 2) 
y_te_cv = np_utils.to_categorical(y_te, 2)

## 1.4 Defining the model

<img src='https://i.imgur.com/w395Yk9.png'>

In [ ]:
# as of now we have vectorized all our features now we will define our model.
# as it is clear from above image that the given model has multiple input layers and hence we have to use functional API
# Please go through - https://keras.io/guides/functional_api/
# it is a good programming practise to define your complete model i.e all inputs , intermediate and output layers at one place.
# while defining your model make sure that you use variable names while defining any length,dimension or size.
#for ex.- you should write the code as 'input_text = Input(shape=(pad_length,))' and not as 'input_text = Input(shape=(300,))'
# the embedding layer for text data should be non trainable
# the embedding layer for categorical data should be trainable
# https://stats.stackexchange.com/questions/270546/how-does-keras-embedding-layer-work
# https://towardsdatascience.com/deep-embeddings-for-categorical-variables-cat2vec-b05c8ab63ac0
#print model.summary() after you have defined the model
#plot the model using utils.plot_model module and make sure that it is similar to the above image

In [30]:
uniq_ss = X_tr['school_state'].nunique()
uniq_tp = X_tr['clean_categories'].nunique()
uniq_cc = X_tr['clean_categories'].nunique()
uniq_csc = X_tr['clean_subcategories'].nunique()
uniq_pgc = X_tr['project_grade_category'].nunique()

In [31]:
embd_size_ss = int(min(np.ceil((X_tr['school_state'].nunique())/2), 50 ))
embd_size_tp = int(min(np.ceil((X_tr['clean_categories'].nunique())/2), 50 ))
embd_size_cc = int(min(np.ceil((X_tr['clean_categories'].nunique())/2), 50 ))
embd_size_csc = int(min(np.ceil((X_tr['clean_subcategories'].nunique())/2), 50 ))
embd_size_pgc = int(min(np.ceil((X_tr['project_grade_category'].nunique())/2), 50 ))

In [35]:
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

In [36]:
checkpoint = ModelCheckpoint(filepath="LSTMmodel_1.h5", monitor="val_auroc", mode="max", save_best_only=True, verbose=2)
earlystop = EarlyStopping(monitor = 'val_auroc', patience = 3, mode="max",min_delta = 0, verbose = True)
logdir = 'logs1'
tensorboard = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [37]:
from keras import regularizers


ip_seq_text_data = Input(shape=(pad_len,), name='essay')
embd_lyr_txt = Embedding(input_dim=vocab_size, output_dim=300, input_length=300 , weights=[embedding_matrix])(ip_seq_text_data)
lstm_lyr_txt = LSTM(128,recurrent_dropout=0.25,kernel_regularizer=regularizers.l2(0.001), return_sequences=True)(embd_lyr_txt)
flatten_text = Flatten()(lstm_lyr_txt)

#catagorical features
school_state = Input(shape=(X_tr_ss.shape[1],), name='school_state')
embd_ss = Embedding(input_dim=uniq_ss, output_dim = embd_size_ss, input_length=1)(school_state)
flatten_school_state = Flatten()(embd_ss)

project_grade_cat = Input(shape=(X_tr_tp.shape[1],), name='project_grade_category')
embd_pgc = Embedding(input_dim=uniq_pgc,output_dim=embd_size_pgc,input_length=1)(project_grade_cat)
flatten_project_grade_cat = Flatten()(embd_pgc)

clean_categories = Input(shape=(X_tr_cc.shape[1],), name='clean_categories')
embd_cc = Embedding(input_dim=uniq_cc,output_dim=embd_size_cc,input_length=1)(clean_categories)
flatten_clean_categories = Flatten()(embd_cc)

clean_subcategories = Input(shape=(X_tr_csc.shape[1],), name='clean_subcategories')
embd_csc = Embedding(input_dim=uniq_csc, output_dim=embd_size_csc, input_length=1)(clean_subcategories)
flatten_clean_subcategories = Flatten()(embd_csc)

teacher_prefix = Input(shape=(X_te_tp.shape[1],), name='teacher_prefix')
embd_tp = Embedding( input_dim=uniq_tp,output_dim=embd_size_tp,input_length=1)(teacher_prefix)
flatten_teacher_prefix = Flatten()(embd_tp)


## Numerical features
numerical_features = Input(shape=(2,) , name="numerical_features")
num_dense_layer = Dense(64, activation='relu', kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(0.001))(numerical_features)

## concatenating all the parallel layers descibed above
concat_parallel_layers = concatenate([flatten_text , flatten_school_state , flatten_project_grade_cat,
flatten_clean_categories , flatten_clean_subcategories , flatten_teacher_prefix , num_dense_layer])


# Dense layer after concatinating
dense_1 = Dense(300, activation="relu", kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(0.001))(concat_parallel_layers)
dropout_lyr_1_1 = Dropout(0.5)(dense_1)
dense_con2 = Dense(256,activation="relu",kernel_initializer="he_normal" ,kernel_regularizer=regularizers.l2(0.001))(dropout_lyr_1_1)
BatchNormalization_1 = BatchNormalization()(dense_con2)
dropout_lyr_1_2 = Dropout(0.3)(BatchNormalization_1)
dense_3 = Dense(96,activation="relu", kernel_initializer="he_normal" ,kernel_regularizer=regularizers.l2(0.001))(dropout_lyr_1_2)


## output
output = Dense(2, activation='softmax', name='output')(dense_3)


In [40]:
## adding input and output to the model
model_1 = Model(inputs=[ip_seq_text_data, school_state, project_grade_cat, clean_categories, clean_subcategories, teacher_prefix, numerical_features], outputs=output)

In [41]:
model_1.summary()

## 1.5 Compiling and fititng your model

In [ ]:
#define custom auc as metric , do not use tf.keras.metrics
# https://stackoverflow.com/a/46844409 - custom AUC reference 1
# https://www.kaggle.com/c/santander-customer-transaction-prediction/discussion/80807  - custom AUC reference 2
# compile and fit your model

In [42]:
# https://stackoverflow.com/questions/41032551/how-to-compute-receiving-operating-characteristic-roc-and-auc-in-keras/46844409#46844409
from sklearn.metrics import roc_auc_score
def auc(y_train, y_pred):
    if len(np.unique(y_train[:,1])) == 1:
        return 0.5
    else:
        return roc_auc_score(y_train, y_pred)

def auroc(y_train, y_pred):
    return tf.py_function(auc, (y_train, y_pred), tf.double)


In [43]:
model_1.compile(loss='categorical_crossentropy',
                optimizer='Adam',
                metrics=[auroc])

In [ ]:
# !rm -rf ./logs

In [44]:
M1 = model_1.fit(
    x = X_tr_fin1,
    y = y_tr_fin,
    batch_size=64,
    epochs=4,
    verbose=2,
    callbacks = [checkpoint,earlystop,tensorboard],
    # validation_split=0.0,
    validation_data=(X_te_fin1, y_te_cv),  
    )

In [ ]:
model_1.save_weights('./m1')

In [53]:
%load_ext tensorboard


In [52]:
!kill 3452

In [54]:
%tensorboard --logdir logs1

In [ ]:
# !rm -rf ./logs

# <font color='red'> Model-2 </font>

Use the same model as above but for 'input_seq_total_text_data' give only some words in the sentance not all the words. Filter the words as below. 

<pre>
1. Fit TF-IDF vectorizer on the Train data <br>
2. Get the idf value for each word we have in the train data. Please go through <a  href='https://stackoverflow.com/questions/23792781/tf-idf-feature-weights-using-sklearn-feature-extraction-text-tfidfvectorizer'>this</a><br>

3. Do some analysis on the Idf values and based on those values choose the low and high threshold value. Because very 
frequent words and very very rare words don't give much information.
Hint - A preferable IDF range is 2-11 for model 2. <br>
4.Remove the low idf value and high idf value words from the train and test data. You can go through each of the
sentence of train and test data and include only those features(words) which are present in the defined IDF range.
5. Perform tokenization on the modified text data same as you have done for previous model.
6. Create embedding matrix for model 2 and then use the rest of the features similar to previous model.
7. Define the model, compile and fit the model.
</pre>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(X_tr['essay'])
essay_vect = vectorizer.transform(X_tr['essay'])

In [ ]:
idf = {'word': vectorizer.get_feature_names() , 'idf_score': vectorizer.idf_}
idf_scr = idf['idf_score']
sorted_idf = idf_scr.sort()


In [ ]:
import matplotlib.pyplot as plt
plt.plot(idf_scr)
plt.grid()
plt.show()


In [ ]:
df= pd.DataFrame(idf)
print(df.shape)
df.head(3)

In [ ]:
df.drop(df[df['idf_score'] < 2.0].index, inplace = True)
df.drop(df[df['idf_score']> 11.0].index, inplace = True)

In [ ]:
print(df.head(2))
print()
print(df.tail(2))

In [ ]:
train_data = np.array(df['word'])

# Tokenize our training data
tokenizer2 = Tokenizer()
tokenizer2.fit_on_texts(train_data)

# Get our training data word index
word_index = tokenizer2.word_index

# Encode training data sentences into sequences
train_sequences = tokenizer2.texts_to_sequences(X_tr['essay'])

# Get max training sequence length
maxlen = 300
# Pad the training sequences
train_padded2= pad_sequences(train_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)
pad_len2 = train_padded.shape[1]

In [ ]:
test_data = X_te['essay']
test_sequences = tokenizer2.texts_to_sequences(X_te['essay'])
test_padded2 = pad_sequences(test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

In [ ]:
train_padded2.shape,test_padded2.shape

In [ ]:
with open('glove_vectors', 'rb') as f:
    glov = pickle.load(f)
    g_words =  set(glov.keys())

In [ ]:
# create a weight matrix for words in training docs
vocab_size2 = len(word_index)+1
embedding_matrix2 = np.zeros((vocab_size2, 300))
for word, i in tokenizer2.word_index.items():
	if word in g_words:
		embedding_vector2 = glov[word]
		embedding_matrix2[i] = embedding_vector2
embedding_matrix2.shape

In [ ]:
X_tr_fin2 = [train_padded2,X_tr_ss,X_tr_tp,X_tr_cc,X_tr_csc,X_tr_pgc,numerical_data_train]
X_te_fin2 = [test_padded2,X_te_ss,X_te_tp,X_te_cc,X_te_csc,X_te_pgc,numerical_data_test]

In [ ]:
checkpoint = ModelCheckpoint(filepath="LSTMmodel_2.h5", monitor="val_auroc", mode="max", save_best_only=True, verbose=2)
earlystop = EarlyStopping(monitor = 'val_auroc', patience = 3, mode="max",min_delta = 0, verbose = 2)
logdir = 'logs2'
tensorboard = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:
from keras import regularizers


ip_seq_text_data = Input(shape=(pad_len2,), name='essay2')
embd_lyr_txt = Embedding(input_dim=vocab_size2, output_dim=300, input_length=300 , weights=[embedding_matrix2])(ip_seq_text_data)
lstm_lyr_txt = LSTM(128,recurrent_dropout=0.25,kernel_regularizer=regularizers.l2(0.001), return_sequences=True)(embd_lyr_txt)
flatten_text = Flatten()(lstm_lyr_txt)

#catagorical features
school_state = Input(shape=(X_tr_ss.shape[1],), name='school_state')
embd_ss = Embedding(input_dim=uniq_ss, output_dim = embd_size_ss, input_length=1)(school_state)
flatten_school_state = Flatten()(embd_ss)

project_grade_cat = Input(shape=(X_tr_tp.shape[1],), name='project_grade_category')
embd_pgc = Embedding(input_dim=uniq_pgc,output_dim=embd_size_pgc,input_length=1)(project_grade_cat)
flatten_project_grade_cat = Flatten()(embd_pgc)

clean_categories = Input(shape=(X_tr_cc.shape[1],), name='clean_categories')
embd_cc = Embedding(input_dim=uniq_cc,output_dim=embd_size_cc,input_length=1)(clean_categories)
flatten_clean_categories = Flatten()(embd_cc)

clean_subcategories = Input(shape=(X_tr_csc.shape[1],), name='clean_subcategories')
embd_csc = Embedding(input_dim=uniq_csc, output_dim=embd_size_csc, input_length=1)(clean_subcategories)
flatten_clean_subcategories = Flatten()(embd_csc)

teacher_prefix = Input(shape=(X_te_tp.shape[1],), name='teacher_prefix')
embd_tp = Embedding( input_dim=uniq_tp,output_dim=embd_size_tp,input_length=1)(teacher_prefix)
flatten_teacher_prefix = Flatten()(embd_tp)


## Numerical features
numerical_features = Input(shape=(2,) , name="numerical_features")
num_dense_layer = Dense(64, activation='relu', kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(0.001))(numerical_features)

## concatenating all the parallel layers descibed above
concat_parallel_layers = concatenate([flatten_text , flatten_school_state , flatten_project_grade_cat,
flatten_clean_categories , flatten_clean_subcategories , flatten_teacher_prefix , num_dense_layer])


# Dense layer after concatinating
dense_1 = Dense(300, activation="relu", kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(0.001))(concat_parallel_layers)
dropout_lyr_2_1 = Dropout(0.5)(dense_1)
dense_con2 = Dense(256,activation="relu",kernel_initializer="he_normal" ,kernel_regularizer=regularizers.l2(0.001))(dropout_lyr_2_1)
BatchNormalization_1 = BatchNormalization()(dense_con2)
dropout_lyr_2_2 = Dropout(0.3)(BatchNormalization_1)
dense_3 = Dense(96,activation="relu", kernel_initializer="he_normal" ,kernel_regularizer=regularizers.l2(0.001))(dropout_lyr_2_2)


## output
output = Dense(2, activation='softmax', name='output')(dense_3)


In [ ]:
## adding input and output to the model
model_2 = Model(inputs=[ip_seq_text_data, school_state, project_grade_cat, clean_categories, clean_subcategories, teacher_prefix, numerical_features], outputs=output)

In [ ]:
model_2.summary()

In [ ]:
model_2.compile(loss='categorical_crossentropy',
                optimizer='Adam',
                metrics=[auroc])

In [ ]:
M2 = model_2.fit(
    x = X_tr_fin2,
    y = y_tr_fin,
    batch_size=64,
    epochs=4,
    verbose=2,
    callbacks = [checkpoint,earlystop,tensorboard],
    # validation_split=0.0,
    validation_data=(X_te_fin2, y_te_cv),  
    )

In [ ]:
model_2.save('D:\Applied_AI\Ass_26_LSTM\weights\m2')

In [ ]:
%tensorboard --logdir logs2

In [ ]:
!rm -rf ./logs

# <font color='red'> Model-3 </font>

<img src='https://i.imgur.com/fkQ8nGo.png'>
ref: https://i.imgur.com/fkQ8nGo.png

In [ ]:
#in this model you can use the text vectorized data from model1 
#for other than text data consider the following steps
# you have to perform one hot encoding of categorical features. You can use onehotencoder() or countvectorizer() for the same.
# Stack up standardised numerical features and all the one hot encoded categorical features
#the input to conv1d layer is 3d, you can convert your 2d data to 3d using np.newaxis
# Note - deep learning models won't work with sparse features, you have to convert them to dense features before fitting in the model.

In [ ]:
train_padded3 = train_padded
test_padded3 = test_padded
train_padded3.shape,test_padded3.shape

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
encoder.fit(X_tr['school_state'].values.reshape(-1,1))
X_tr_ss_ohe = encoder.transform(X_tr['school_state'].values.reshape(-1,1))
X_te_ss_ohe = encoder.transform(X_te['school_state'].values.reshape(-1,1))
X_tr_ss_ohe.shape

In [ ]:
encoder.fit(X_tr['teacher_prefix'].values.reshape(-1,1))
X_tr_tp_ohe = encoder.transform(X_tr['teacher_prefix'].values.reshape(-1,1))
X_te_tp_ohe = encoder.transform(X_te['teacher_prefix'].values.reshape(-1,1))

In [ ]:
encoder.fit(X['clean_categories'].values.reshape(-1,1))
X_tr_cc_ohe = encoder.transform(X_tr['clean_categories'].values.reshape(-1,1))
X_te_cc_ohe = encoder.transform(X_te['clean_categories'].values.reshape(-1,1))

In [ ]:
encoder.fit(X['clean_subcategories'].values.reshape(-1,1))
X_tr_csc_ohe = encoder.transform(X_tr['clean_subcategories'].values.reshape(-1,1))
X_te_csc_ohe = encoder.transform(X_te['clean_subcategories'].values.reshape(-1,1))

In [ ]:
encoder.fit(X['project_grade_category'].values.reshape(-1,1))
X_tr_pgc_ohe = encoder.transform(X_tr['project_grade_category'].values.reshape(-1,1))
X_te_pgc_ohe = encoder.transform(X_te['project_grade_category'].values.reshape(-1,1))

In [ ]:
print('Encoded train school_state shape',X_tr_ss_ohe.shape)
print('Encoded train teacher_prefix shape',X_tr_tp_ohe.shape)
print('Encoded train clean_categories shape',X_tr_cc_ohe.shape)
print('Encoded train clean_subcategories shape',X_tr_csc_ohe.shape)
print('Encoded train project_grade_category shape',X_tr_pgc_ohe.shape)

In [ ]:
print('Encoded test school_state shape',X_te_ss_ohe.shape)
print('Encoded test teacher_prefix shape',X_te_tp_ohe.shape)
print('Encoded test clean_categories shape',X_te_cc_ohe.shape)
print('Encoded test clean_subcategories shape',X_te_csc_ohe.shape)
print('Encoded test project_grade_category shape',X_te_pgc_ohe.shape)

In [ ]:
X_tr_np3 = X_tr_np
X_te_np3 = X_te_np
X_tr_np3.shape,X_te_np3.shape

In [ ]:
X_tr_price3 = X_tr_price
X_te_price3 = X_te_price
X_tr_price3.shape,X_te_price3.shape

In [ ]:
X_tr_ss_ohe.shape, X_tr_tp_ohe.shape , X_tr_cc_ohe.shape, X_tr_csc_ohe.shape , X_tr_pgc_ohe.shape , X_tr_np3.shape , X_tr_price3.shape

In [ ]:
from scipy.sparse import hstack
X_tr_cat_num = hstack((X_tr_ss_ohe,X_tr_tp_ohe,X_tr_cc_ohe,X_tr_csc_ohe,X_tr_pgc_ohe,X_tr_np3,X_tr_price3)).todense()
X_te_cat_num = hstack((X_te_ss_ohe,X_te_tp_ohe,X_te_cc_ohe,X_te_csc_ohe,X_te_pgc_ohe,X_te_np3,X_te_price3)).todense()
X_tr_cat_num.shape,X_te_cat_num.shape,

In [ ]:
# https://www.geeksforgeeks.org/insert-a-new-axis-within-a-numpy-array/
X_tr_cat_num3d = X_tr_cat_num[...,np.newaxis]
X_te_cat_num3d = X_te_cat_num[...,np.newaxis]
X_tr_cat_num3d.shape, X_te_cat_num3d.shape

In [ ]:
X_tr_fin3 = [train_padded3,X_tr_cat_num3d]
X_te_fin3 = [test_padded3,X_te_cat_num3d]
len(X_tr_fin3),len(X_tr_fin3[0]),len(X_tr_fin3[0][0])

In [ ]:
checkpoint = ModelCheckpoint(filepath="./LSTMmodel_3.h5", monitor="val_auroc", mode="max", save_best_only=True, verbose=2)
earlystop = EarlyStopping(monitor = 'val_auroc', patience = 5, mode="max",min_delta = 0, verbose = 2)
logdir = 'logs3'
tensorboard = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:
pad_len

In [ ]:
ip_seq_text_data3 = Input(shape=(pad_len,), name='essay3')
embd_lyr_txt = Embedding(input_dim=vocab_size, output_dim=300, input_length=300 , weights=[embedding_matrix])(ip_seq_text_data3)
lstm_lyr_txt = LSTM(128,recurrent_dropout=0.25,kernel_regularizer=regularizers.l2(0.001), return_sequences=True)(embd_lyr_txt)
flatten_text = Flatten()(lstm_lyr_txt)

cat_num  = Input(shape=(514,1), name='cat and num')
conv3_1 = Conv1D(128 ,3 ,activation='relu', kernel_initializer=he_normal(seed=None) , padding='valid')(cat_num)
conv3_2 = Conv1D(64 ,3 ,activation='relu', kernel_initializer=he_normal(seed=None) , padding='valid')(conv3_1)
cat_num_flatten_3 = Flatten()(conv3_2)

## combining the parallel branches
concat_m3 = concatenate([flatten_text, cat_num_flatten_3])

dense_lyr_3_1 = Dense(128, activation="relu", kernel_initializer="he_normal", kernel_regularizer=regularizers.l2(0.001))(concat_m3)

dropout_lyr_3_1 = Dropout(0.2)(dense_lyr_3_1)

dense_lyr_3_2 = Dense(256,activation="relu",kernel_initializer="he_normal" ,kernel_regularizer=regularizers.l2(0.001))(dropout_lyr_3_1)

dropout_lyr_3_2 = Dropout(0.2)(dense_lyr_3_2)

dense_lyr_3_3 = Dense(128, activation="relu", kernel_initializer="he_normal", kernel_regularizer=regularizers.l2(0.001))(dropout_lyr_3_2)

output = Dense(2, activation='softmax', name='output')(dense_lyr_3_3)

In [ ]:
ip_seq_text_data3.shape,cat_num.shape

In [ ]:
model_3 = Model(inputs=[ip_seq_text_data3, cat_num],outputs=output)

In [ ]:
model_3.summary()

In [ ]:
model_3.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=[auroc])

In [ ]:
model_3.fit(
    x = X_tr_fin3,
    y = y_tr_fin,
    batch_size=128,
    epochs =4,
    verbose=2,
    validation_data=(X_te_fin3, y_te_cv),
    callbacks=[checkpoint,earlystop,tensorboard]
    )

In [ ]:
model_3.save('D:\Applied_AI\Ass_26_LSTM\weights\m3')

In [ ]:
%tensorboard --logdir logs